In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ammaarahmad1999/Naive-Bias-Spam-Classifier/main/SMSSpamCollection', sep='\t', header=None, names =['Label', 'SMS'])

In [3]:
print(df.head(10))

  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
5  spam  FreeMsg Hey there darling it's been 3 week's n...
6   ham  Even my brother is not like to speak with me. ...
7   ham  As per your request 'Melle Melle (Oru Minnamin...
8  spam  WINNER!! As a valued network customer you have...
9  spam  Had your mobile 11 months or more? U R entitle...


In [4]:
#df = df.sample(frac=1, random_state=50)

In [5]:
print(df['Label'].value_counts())

ham     4825
spam     747
Name: Label, dtype: int64


In [6]:
length = len(df)
print(length)

5572


In [7]:
def multinomial_train(df_train):

  vocabulary = set()
  for sms in df_train['SMS']:
    for word in sms:
        vocabulary.add(word)

  word_counts_per_sms = {unique_word: [0] * len(df_train['SMS']) for unique_word in vocabulary}

  for index, sms in enumerate(df_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1  
  
  word_counts = pd.DataFrame(word_counts_per_sms)
  df_train = pd.concat([df_train, word_counts], axis=1)

  # Isolating spam and ham messages first
  spam_messages = df_train[df_train['Label'] == 'spam']
  ham_messages = df_train[df_train['Label'] == 'ham']

  # P(Spam) and P(Ham)
  p_spam = len(spam_messages) / len(df_train)
  p_ham = len(ham_messages) / len(df_train)

  # N_Spam
  n_words_per_spam_message = spam_messages['SMS'].apply(len)
  n_spam = n_words_per_spam_message.sum()

  # N_Ham
  n_words_per_ham_message = ham_messages['SMS'].apply(len)
  n_ham = n_words_per_ham_message.sum()

  # N_Vocabulary
  n_vocabulary = len(vocabulary)
  print(f'\tLength of vocabulary = {n_vocabulary}')

  # Laplace smoothing
  alpha = 1

  # Initiate parameters
  parameters_spam = {unique_word:0 for unique_word in vocabulary}
  parameters_ham = {unique_word:0 for unique_word in vocabulary}

  # Calculate parameters
  for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham  
  
  return p_spam, p_ham, parameters_spam, parameters_ham, vocabulary

In [8]:
def multivariate_train(df_train):

  vocabulary = set()
  for sms in df_train['SMS']:
    for word in sms:
        vocabulary.add(word)

  word_counts_per_sms = {unique_word: [0] * len(df_train['SMS']) for unique_word in vocabulary}

  for index, sms in enumerate(df_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1  
  
  word_counts = pd.DataFrame(word_counts_per_sms)
  df_train = pd.concat([df_train, word_counts], axis=1)

  # Isolating spam and ham messages first
  spam_messages = df_train[df_train['Label'] == 'spam']
  ham_messages = df_train[df_train['Label'] == 'ham']

  # P(Spam) and P(Ham)
  p_spam = len(spam_messages) / len(df_train)
  p_ham = len(ham_messages) / len(df_train)

  # N_Spam
  n_words_per_spam_message = spam_messages['SMS'].apply(len)
  n_spam = n_words_per_spam_message.sum()

  # N_Ham
  n_words_per_ham_message = ham_messages['SMS'].apply(len)
  n_ham = n_words_per_ham_message.sum()

  # N_Vocabulary
  n_vocabulary = len(vocabulary)
  print(f'\tLength of vocabulary = {n_vocabulary}')

  # Laplace smoothing
  alpha = 1

  # Initiate parameters
  parameters_spam = {unique_word:0 for unique_word in vocabulary}
  parameters_ham = {unique_word:0 for unique_word in vocabulary}

  # Calculate parameters
  for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + 1) / (len(spam_messages) + 2)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + 1) / (len(ham_messages) + 2)
    parameters_ham[word] = p_word_given_ham  
  
  return p_spam, p_ham, parameters_spam, parameters_ham, vocabulary

In [9]:
def test_model(df_test, p_spam, p_ham, parameters_spam, parameters_ham, vocabulary):

    correct = 0
    total = df_test.shape[0]

    for i in range(len(df_test)):
        
        p_spam_given_message = math.log(p_spam)
        p_ham_given_message = math.log(p_ham)
        message = df_test.iloc[i, 1]
        label = df_test.iloc[i, 0]

        '''for word in vocabulary:
          if word in message:
            
            p_spam_given_message += math.log(parameters_spam[word])
            p_spam_given_message += math.log(1 - parameters_spam[word])
            
            p_ham_given_message += math.log(parameters_ham[word])
            p_ham_given_message += math.log(1 - parameters_ham[word])'''
        
        for word in message:
          if word in parameters_spam:
            p_spam_given_message += math.log(parameters_spam[word])
            p_spam_given_message += math.log(1 - parameters_spam[word])
          if word in parameters_ham:
            p_ham_given_message += math.log(parameters_ham[word])
            p_ham_given_message += math.log(1 - parameters_ham[word])
        

        if p_ham_given_message >= p_spam_given_message:
            predicted = 'ham'
        else:
            predicted = 'spam'

        if (predicted == label):
            correct += 1

    return correct/total

In [10]:
# Divide the data set into 5 equal chunks
size = int(len(df)/5)
df0 = df[:size]
df1 = df[size:2*size]
df2 = df[2*size:3*size]
df3 = df[3*size:4*size]
df4 = df[4*size:]

In [11]:
multinomial_accuracy = []
multivariate_accuracy = []

for i in range(5):
    print('Iteration-', i+1, ':', sep='')

    if (i == 0):
      df_train = pd.concat([df1, df2, df3, df4])
      df_test = df0
    if (i == 1):
      df_train = pd.concat([df0, df2, df3, df4])
      df_test = df1
    if (i == 2):
      df_train = pd.concat([df0, df1, df3, df4])
      df_test = df2
    if (i == 3):
      df_train = pd.concat([df0, df1, df2, df4])
      df_test = df3
    if (i == 4):
      df_train = pd.concat([df0, df1, df2, df3])
      df_test = df4
    
    print('\tSize of training set =', len(df_train))
    print('\tSize of testing set =', len(df_test))

    #CLeaning the dataset
    df_train['SMS'] = df_train['SMS'].str.replace('\W', ' ') # Removes punctuation
    df_train['SMS'] = df_train['SMS'].str.lower()            # Convert to Lowercase
    df_train['SMS'] = df_train['SMS'].str.split()            # Split into words
    
    # Train, test and get the accuracy of multinomial model
    p_spam, p_ham, parameters_spam, parameters_ham, vocabulary = multinomial_train(df_train)
    acc  = test_model(df_test, p_spam, p_ham, parameters_spam, parameters_ham, vocabulary)

    print(f'\tAccuracy of the multinomial model = {acc}')
    print()
    multinomial_accuracy.append(acc)

    # Train, test and get the accuracy of multinomial model
    p_spam, p_ham, parameters_spam, parameters_ham, vocabulary = multivariate_train(df_train)
    acc  = test_model(df_test, p_spam, p_ham, parameters_spam, parameters_ham, vocabulary)

    print(f'\tAccuracy of the multivariate model = {acc}')
    print()
    multivariate_accuracy.append(acc)

# Print the mean accuracies of the models
print('Mean acccuracy of multinomial model =', np.mean(multinomial_accuracy))
print('Mean acccuracy of multivariate model =', np.mean(multivariate_accuracy))

Iteration-1:
	Size of training set = 4458
	Size of testing set = 1114
	Length of vocabulary = 7806
	Accuracy of the multinomial model = 0.8491921005385996

	Length of vocabulary = 7806
	Accuracy of the multivariate model = 0.32046678635547576

Iteration-2:
	Size of training set = 4458
	Size of testing set = 1114
	Length of vocabulary = 7762
	Accuracy of the multinomial model = 0.8734290843806104

	Length of vocabulary = 7762
	Accuracy of the multivariate model = 0.23159784560143626

Iteration-3:
	Size of training set = 4458
	Size of testing set = 1114
	Length of vocabulary = 7775
	Accuracy of the multinomial model = 0.8752244165170556

	Length of vocabulary = 7775
	Accuracy of the multivariate model = 0.414721723518851

Iteration-4:
	Size of training set = 4458
	Size of testing set = 1114
	Length of vocabulary = 7789
	Accuracy of the multinomial model = 0.8572710951526032

	Length of vocabulary = 7789
	Accuracy of the multivariate model = 0.5385996409335727

Iteration-5:
	Size of train